In [181]:
#bibliothèque
from sklearn.linear_model import LinearRegression
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import random

# Fonction pour changer la couleur du texte
def couleur(valeur):
    if valeur < 0:
        return f'<font color="red">{valeur}</font>'
    else:
        return f'<font color="green">{valeur}</font>'

Pour obtenir une liste avec tout les tags des entreprises du CAC40 :

In [182]:
# Chemin du dossier
dossier = "data-fr/Data_CAC40"
fichiers = os.listdir(dossier)
list_tags = []
for i in fichiers:
    chaine = i
    chaine_sans_dataframe = chaine.replace("dataframe_", "")
    second = chaine_sans_dataframe.replace(".csv","")
    list_tags.append(second)

print(list_tags)
list_tags = ['SU.PA', 'ORA.PA', 'ACA.PA', 'STM.PA', 'CAP.PA', 'DSY.PA', 'MC.PA', 'MT.AS', 'RI.PA', 'ENGI.PA', 'KER.PA', 'BN.PA', 'LR.PA', 'BNP.PA', '^FCHI', 'AIR.PA', 'SAN.PA', 'SGO.PA', 'GLE.PA', 'OR.PA', 'ML.PA', 'CA.PA', 'SAF.PA', 'AI.PA']


['SU.PA', 'ORA.PA', 'ACA.PA', 'STM.PA', 'CAP.PA', 'DSY.PA', 'MC.PA', 'MT.AS', 'RI.PA', 'ENGI.PA', 'KER.PA', 'BN.PA', 'LR.PA', 'BNP.PA', '^FCHI', 'AIR.PA', 'SAN.PA', 'SGO.PA', 'GLE.PA', 'OR.PA', 'ML.PA', 'CA.PA', 'SAF.PA', 'AI.PA']


Class portefeuille d'action:

In [195]:
class portefeuille:
    def __init__(self, noms_titres, quantites, time_achat) -> None:
        self.noms_titres = noms_titres
        self.quantites = quantites
        self.time = time_achat
        #On importe les données
        self.data = []
        self.prix = []
        for i in self.noms_titres:
            path = "data-fr/Data_CAC40/dataframe_"+i+".csv"
            df = pd.read_csv(path)
            self.data.append(df)
            #print(i, "chargé")
        #On récupère le prix de fermeture des actions à l'instant t = time_achat
        self.recuperer_prix(self.time)
        #Création du dataframe de notre portefeuille
        self.complete_actions_df()

    def __str__(self) -> str:
        return f'{self.actions}'
    
    def get_noms_titres(self):
        return self.noms_titres

    def recuperer_prix(self, t):
        for i in self.data:
            self.prix.append(i['Close'][t])

            
    def get_data(self):
        return self.data
    
    def complete_actions_df(self):
        self.actions_temp = {'Prix achat à t = ' + str(self.time) :self.prix,
                        'Quantite': self.quantites}
        self.actions = pd.DataFrame(self.actions_temp, index=self.noms_titres)
        self.actions['Valeur des actions en $'] = self.actions.apply(lambda row: row['Prix achat à t = ' + str(self.time)] * row['Quantite'], axis=1)

    def add_predic(self, t, mod):
            predict = []
            for i in mod.models:
                predict.append(i.predict(np.array([t]).reshape(1,1))[0])
            self.actions['Évolution du prix en %'] = 100*(predict-self.actions['Prix achat à t = ' + str(self.time)])/self.actions['Prix achat à t = ' + str(self.time)]
            self.actions['Estimation du prix de unitaire à t = '+str(t)] = predict
            self.actions['Estimation de la valeur des actions à t = '+str(t)] = self.actions.apply(lambda row: row['Estimation du prix de unitaire à t = '+str(t)] * row['Quantite'], axis=1)
            #self.couleur_evolution()
            self.actions['Score de la prédiction'] = self.scores
    def afficher_resultats(self):
        return self.actions.head(len(self.actions))
    
    def couleur_evolution(self):
        self.actions['Évolution du prix en %'] = self.actions['Évolution du prix en %'].apply(lambda x: couleur(x))

    def set_scores(self, scores):
        self.scores = scores

### Les models

In [196]:
class lr_models:
    #period = période d'apprentissage => inputs
    def __init__(self, portefeuille, period, learning_rate) -> None:
        self.mon_portefeuille = portefeuille 
        self.noms_titres = portefeuille.get_noms_titres()
        self.m = period
        self.learning_rate = learning_rate
        self.scores = []

    def create_models(self):
        self.models = []
        for i in self.mon_portefeuille.get_data():
            self.models.append(self.get_model(i))


    def get_model(self, df_titre):
        df = df_titre.head(self.m)
        self.Y = df['Close']
        self.X = np.arange(self.m).reshape(self.m,1)
        model = LinearRegression()
        model.fit(self.X,self.Y)
        self.scores.append(model.score(self.X,self.Y))
        self.mon_portefeuille.set_scores(self.scores)
        #print(model.score(self.X, self.Y))
        return model


In [197]:
#Quelles actions on veut dans notre portefeuille d'actions et quelles sont les quantités? 
liste_tags_entreprises = list_tags
quantites = [random.randint(1, 10) for _ in range(len(liste_tags_entreprises))]
buy_order = 0
time_target = 200 #Pour quand on souhaite effectuer des prédictions
#Paramètres de l'apprentissage :
learning_period = 100
learning_rate = 0.01

Création de mon portefeuille, et de mes modèles d'apprentissages + Ajouts des prédictions

In [198]:
portefeuille_maxime = portefeuille(liste_tags_entreprises,quantites, buy_order)
model = lr_models(portefeuille_maxime, learning_period, learning_rate)
model.create_models()
portefeuille_maxime.add_predic(time_target,model)

In [199]:
portefeuille_maxime.add_predic(time_target,model)

In [200]:
portefeuille_maxime.actions.head()

,Prix achat à t = 0,Quantite,Valeur des actions en $,Évolution du prix en %,Estimation du prix de unitaire à t = 200,Estimation de la valeur des actions à t = 200,Score de la prédiction
SU.PA,29.081612,1,29.081612,15.860335,33.694053,33.694053,0.343671
ORA.PA,7.673621,10,76.736207,-13.702020,6.622180,66.221796,0.192507
ACA.PA,7.259819,6,43.558911,-25.958321,5.375292,32.251749,0.108057
STM.PA,3.978039,9,35.802347,29.605029,5.155738,46.401642,0.337022
CAP.PA,25.837217,6,155.023304,36.434455,35.250867,211.505200,0.631840
